In [181]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography import x509
from cryptography.x509.oid import NameOID
import array

In [182]:
def generateKey():
    key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    return key

In [183]:
def persistKey(key,filePath="key.pem",passphrase=b"mypassword"):
    with open(filePath, "wb") as f:
             f.write(key.private_bytes(
             encoding=serialization.Encoding.PEM,
             format=serialization.PrivateFormat.TraditionalOpenSSL,
             encryption_algorithm=serialization.BestAvailableEncryption(passphrase),
        ))

In [184]:
def getKeyPair(filePath="key.pem",passphrase= b"mypassword"):
    with open(filePath, "rb") as key_file:
               privateKey = serialization.load_pem_private_key(
               key_file.read(),
               password=passphrase,
               backend=default_backend()
    )
    publicKey = privateKey.public_key()
    return (privateKey, publicKey)
    

In [185]:
def signMessage(message,privateKey):
    signature = privateKey.sign(
            message,
            padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
              salt_length = padding.PSS.MAX_LENGTH
            ),
           hashes.SHA256()
    )
    return signature

In [186]:
#verify signature
def verifySignature(signature,message, publicKey):
    try:
        publicKey.verify(
                signature,
                message,
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH
               ),
                hashes.SHA256()
        )
        return True
    except:
        return False

In [187]:
key = generateKey()


In [188]:
persistKey(key)

In [190]:
privateKey,publicKey =  getKeyPair()

In [191]:
message = b"A message I want to sign"
signature = signMessage(message,privateKey)

In [192]:
verifySignature(signature, message, publicKey)

True

In [193]:
def encryptMessage(message,publicKey):
    ciphertext = publicKey.encrypt(
            message,
            padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
         )
       )
    return ciphertext

In [194]:
ciphertext

b'\x06\x9c\x8d*c\xcc}\x84\x1a;8Jui\x10\x8b\x8e\x9f\x8cV\xf5WO\xf3\x18\xf9\xe62\xa0\xe3\xba\x04\xab|>#\xf1\xa4\xe2\xe4x\x16\x1d\xa2|\xe6\xd9\xd8O\xf27\x94)X\xe7u$\xc52\xf2\xe2\x1fLy\xe1\x9cX\x10\x137%\x98\xac\x04c\xd1nl\x8bt\xbb\x1b\xc9\xb8\xbd\xa1\xc93\xae\x94E(5_\xe2\xc5ESg\xd4\xf0O\xde\t\xb3\x9b\xc3n\xd2\x04\x94\xe3O\x01D=\x00\xbd\x88\x1f\xab\x86$\xf9lq\xed\x8fE\xc4X\xd2B\xa8\xeaz\xb1aqeN\xd4\xbc\x90\xa8\x89o\xf6\xccO\xda%\x0e\xa6r\xa1\x92\x95\xb4\xb3[\xf0d\xd0\xecp\xab\x8a\xf8,\x8a\\\xa3\xf7T\xd6\xfc\x08\xe2\xe5\xe8\xc4Q\xc6\x17*\x8aX\xc6\xfb\xce9\xf3\x94-,~3Mrh\xe1\xde\xf5\xa9\x82\xea\x8f>\xf9mly\xfe3\x83[b\xa5\x97\xc0.N\ry\x03\x02\x15\xf4L~=\xa4\xc4\xd8\x85\xe4`>q\xb3\xa4\xdb\x0e\xd1"8/3\x13\x1e\xce\xe3\xae\xb7\x9c'

In [195]:
def decryptMessage(ciphertext, privateKey):
    return privateKey.decrypt(
            ciphertext,
            padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
            )
    )

In [196]:
plaintext = b"I am varun"
ciphertext = encryptMessage(plaintext,publicKey)

In [197]:
decryptMessage(ciphertext,privateKey)

b'I am varun'

In [198]:
def serializePublicKey(publicKey):
    return publicKey.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

In [177]:
serializePublicKey(publicKey)

b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAqbUsVpHB7ICHJ+nkfrNQ\n0ts7kft6rWRq6dCj14ZrL1Elcan0vUUx6g2xF1joY+AnIEOzClRlE3PZDCSPAfCS\n9wtIYAVy+oWVxWEVQ/eXlO9WAm6iE4hmZ+R3hpeOWlYCYTTdTHQCiLkx2E5SRqW4\nXAQmem3a4ajfmudY5yLjQ8DaMey9Wh4+PFRJhZEtnubaBb0zORJRA/H5j0VynT93\nlWJZ05prR87Io6ANLLWZGzaq/3kXj3/Y5Y+uA/6hYXxZbwR70KFCnFDwoPnKpl+X\nhhJJzIw4XmnsAU/CT3TY2cCnZZ3DNwymsVUybMCK747J+pAXxhBlZJSivuRxZwyF\nqQIDAQAB\n-----END PUBLIC KEY-----\n'

In [201]:
def encodeData(data):
    return str(data).encode('utf-8')

In [202]:
dataToEncode = ["acjn34",serializePublicKey(publicKey)]
encodeData(dataToEncode)

b"['acjn34', b'-----BEGIN PUBLIC KEY-----\\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAwrteu6s9Um9+NQ/StkPf\\nW+1pIT8KYtNrtmg0PdO5zTnkbEDBo+UAmpBf636fKOICRxPQ438UaxrFKmtDEYib\\ncInOGmw8fZ4LKQPxkRo4dlXK3cIpAG4Lzp0tqR6Ll1k68t0i1CF/O8abN98vxySC\\nyPBIuMPKGFsShqU1MQshGqqp3HUzXHi4fBbd6igGO7H4iAYMRbNjGeZ3XfxUvQG0\\nrUgg5vOyfn5P2AoY9uHGgW313CcEbEh/QQ0QOCuRo11UTRm+h+84l3UKSQUSkFOa\\nVnQdEN2SZOiHopC/Z/7JAZax8nMU/SkgUQ/01kTKYYYJBjo3+nBC239CcKCLqeVl\\n3QIDAQAB\\n-----END PUBLIC KEY-----\\n']"

In [209]:
def decodeBytes(bytes,toList=False):
    try:
        returnVal = bytes.decode('utf-8')
        print(returnVal)
        return returnVal
    except:
        raise
#         return False

In [213]:
list(dataToEncode)[1]

b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAwrteu6s9Um9+NQ/StkPf\nW+1pIT8KYtNrtmg0PdO5zTnkbEDBo+UAmpBf636fKOICRxPQ438UaxrFKmtDEYib\ncInOGmw8fZ4LKQPxkRo4dlXK3cIpAG4Lzp0tqR6Ll1k68t0i1CF/O8abN98vxySC\nyPBIuMPKGFsShqU1MQshGqqp3HUzXHi4fBbd6igGO7H4iAYMRbNjGeZ3XfxUvQG0\nrUgg5vOyfn5P2AoY9uHGgW313CcEbEh/QQ0QOCuRo11UTRm+h+84l3UKSQUSkFOa\nVnQdEN2SZOiHopC/Z/7JAZax8nMU/SkgUQ/01kTKYYYJBjo3+nBC239CcKCLqeVl\n3QIDAQAB\n-----END PUBLIC KEY-----\n'

In [214]:
from Crypto.PublicKey import RSA

In [215]:
RSA.importKey(list(dataToEncode)[1])

<_RSAobj @0x10c047a58 n(2048),e>

In [216]:
publicKey